In [22]:
import os
import cv2
from PIL import Image
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [23]:
TRAIN_PATH = "./data/train/"
imgnames = [filename for filename in os.listdir(TRAIN_PATH)]


In [24]:
csv_path = "./data/train.csv"
df = pd.read_csv(csv_path)
df = df.dropna().reset_index(drop=True)
df.head()

,ImageId_ClassId,EncodedPixels
0,0002cc93b.jpg_1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0007a71bf.jpg_3,18661 28 18863 82 19091 110 19347 110 19603 11...
2,000a4bcdd.jpg_1,37607 3 37858 8 38108 14 38359 20 38610 25 388...
3,000f6bf48.jpg_4,131973 1 132228 4 132483 6 132738 8 132993 11 ...
4,0014fce06.jpg_3,229501 11 229741 33 229981 55 230221 77 230468...


In [25]:
file_df = df["ImageId_ClassId"].str.split("_", expand=True).rename(columns={0:"ImageId", 1:"class"})
new_df = pd.merge(file_df, df, right_index=True, left_index=True)[["ImageId", "class", "EncodedPixels"]]
new_df.head()

,ImageId,class,EncodedPixels
0,0002cc93b.jpg,1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0007a71bf.jpg,3,18661 28 18863 82 19091 110 19347 110 19603 11...
2,000a4bcdd.jpg,1,37607 3 37858 8 38108 14 38359 20 38610 25 388...
3,000f6bf48.jpg,4,131973 1 132228 4 132483 6 132738 8 132993 11 ...
4,0014fce06.jpg,3,229501 11 229741 33 229981 55 230221 77 230468...


In [26]:
new_df.iloc[1,2]

'18661 28 18863 82 19091 110 19347 110 19603 110 19859 110 20115 110 20371 110 20627 110 20883 110 21139 110 21395 110 21651 110 21962 55 293125 251 293381 251 293637 251 293893 251 294149 251 294405 251 294661 251 294917 251 295173 251 295429 251 295685 251 295941 251 296197 251 296453 251 296709 251 296965 251 297221 251 297477 251 297733 251 297989 251 298245 251 298564 188 298945 63'

In [27]:
def rle_to_pixels(str_rle):
    pixels = []
    result = [int(value) for value in str_rle.split(" ")]
    result_list = np.array(result).reshape(-1,2).tolist()
    for start, end in result_list:
        pixels.extend(list(range(start-1, start+end-1)))
    return pixels

In [ ]:
new_df["list_pixels"] = new_df.EncodedPixels.map(rle_to_pixels)

In [ ]:
new_df.head()

In [ ]:
def rle_to_mask(filepath, df):
    img = cv2.imread(filepath)
    width, height, channel = img.shape
    mask = np.zeros((width,height), dtype=np.int16).flatten()
    for index, rows in df.iterrows(): 
        mask[rows["list_pixels"]] = rows["class"]
    return np.stack( (mask.reshape(width,height),) * 3, axis=-1) 

In [ ]:
filepath_dict = {filename : os.path.join(TRAIN_PATH,filename) for filename in imgnames}
result = rle_to_mask(filepath_dict["0025bde0c.jpg"], new_df[new_df["ImageId"] == "0025bde0c.jpg"])

In [ ]:
from data_utils.data_loader import \
    get_pairs_from_paths, DATA_LOADER_SEED, class_colors, DataLoaderError


In [ ]:
def _get_colored_segmentation_image( seg, colors, n_classes, do_augment=False):
    """ Return a colored segmented image """
    seg_img = np.zeros_like(seg)

    for c in range(n_classes):
        seg_img[:, :, 0] += ((seg[:, :, 0] == c) *
                            (colors[c][0])).astype('uint8')
        seg_img[:, :, 1] += ((seg[:, :, 0] == c) *
                            (colors[c][1])).astype('uint8')
        seg_img[:, :, 2] += ((seg[:, :, 0] == c) *
                            (colors[c][2])).astype('uint8')

    return seg_img

In [ ]:
new_seg = _get_colored_segmentation_image(result, class_colors, 11)
plt.imshow(new_seg, interpolation='nearest')
plt.show()